#### Lovasz-Softmax Loss
- "The Lovasz-Softmax loss: A tractable surrogate for optimize of cross-over-union measure in neural network"(https://arxiv.org/abs/1705.08790)

#### 코드 Step-by-Step

In [ ]:
# 아래 설명은 아래 링크를 참조 하였음
# https://www.kaggle.com/c/tgs-salt-identification-challenge/discussion/67791

# Step1) 예측 에러를 계산
'''
에러 벡터는 예측이 False이면 양수이고, 예측이 True이면 음수이다.
올바른 예측에서 margin 이 1보다 낮으면 오류로 간주된다.
'''
signs = 2. * labels.float() - 1.
errors = (1. - logits * Variable(signs))
error_sorted, perm = torch.sort(errors, dim=0, desending=True)

# Step2) 정렬된 에러를 기준으로 Lovasz Extension
'''
벡터 jaccard에는 정렬된 오류와 관련된 jaccard 인덱스를 포함한다. 
값은 0 ~ jaccard index 사이이다.
'''
gts = gt_sorted.sum()
intersection = gts - gt_sorted.float().cumsum(0)
union = gts + (1 - gt_sorted).float().cumsum(0)
jaccard = 1. - intersection / union

# Step3) 정렬된 에러를 기준으로 Lovasz Extension의 기울기
'''
위에서 구한 벡터의 gradient 계산
해당 벡터는 오류가 Jaccard 지수 변화에 미치는 영향을 나타낸다.
Jaccard 지수에 가장 큰 불이익을 주는 오류를 최소화한다.
'''
jaccard[1:p] = jaccard[1:p] - jaccard[0:-1]

# Step4) 로스를 계산
'''
에러 벡터와 Lovasz extension 기울기 와의 dot product로 실제 로스가 계산된다.
'''
loss = torch.dot(F.relu(errors_sorted), Variable(grad))

#### 이하 [논문](https://arxiv.org/abs/1705.08790) 서론 참고

The Jaccard index, also referred to as the intersection-over-union score, is commonly employed in the evaluation of image segmentation results given its perceptual qualities, scale invariance - which lends appropriate relevance to small objects, and appropriate counting of false negatives, in comparison to per-pixel losses.  
--> IoU 점수라고도 하는 Jaccard 계수는 이미지 세그멘테이션 평가에 일반적으로 사용된다.   
--> Perception 퀄리티, Scale-Invariance(작은 객체, 큰 객체 무관한 성능 유도), False-Negative 에 대한 적절한 Counting을 제공??

We present a method for direct optimization of the mean intersection-over-union loss in neural networks, in the context of semantic image segmentation, based on the convex Lovász extension of submodular losses.   
--> 이미지 분할에서 서브모듈 로스(?)의 Convex Lovasz 확장을 기반으로, 신경망의 평균 IoU Loss을 직접 최적화 하는 방법 제시  
--> (서브 모듈 로스와 Convex Lovasz 확장이 무엇을 말하는 건지??)  
  
The loss is shown to perform better with respect to the Jaccard index measure than the traditionally used cross-entropy loss. We show quantitative and qualitative differences between optimizing the Jaccard index per image versus optimizing the Jaccard index taken over an entire dataset.  
--> 전통적으로 사용되는 크로스 엔트로피 로스보다, IoU 지수 측정에서 더 나은 성능 보인다.  
--> 이미지 당 IoU 인덱스를 최적화 하는 것과 전체 데이터 셋에서 사용되는 IoU 인덱스를 최적화 하는 질적, 양적 차이를 보여준다.  
  
We evaluate the impact of our method in a semantic segmentation pipeline and show substantially improved intersection-over-union segmentation scores on the Pascal VOC and Cityscapes datasets using state-of-the-art deep learning segmentation architectures.  
--> Pascal VOC, Cityscapes 데이터 셋에서 향상된 IoU 계수를 보여준다.

자카드 (Jaccard Index) 계수
- 두 집합의 교집합을 합집합으로 나눈 값


![img.png](https://miro.medium.com/v2/resize:fit:640/format:webp/1*TG5hwQ3lYaKSCEBlvMlPPw.png)